Both BFV (with batchencoder)and CKKS can perform (slot-wise) vectorized computation.  
In addition to that, it can *rotate* the vector cyclically. (shift the index)

Skip BFV example

In [1]:
import seal
import numpy as np

In [26]:
class scheme():
    def __init__(self, poly_modulus_degrees, coeff_mod_bits, scheme="ckks"):
        self.parms = seal.EncryptionParameters(getattr(seal.scheme_type, scheme))
        self.parms.set_poly_modulus_degree(poly_modulus_degrees)
        self.parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degrees,
                                                         coeff_mod_bits))

        self.context = seal.SEALContext(parms)

        self.keygen = seal.KeyGenerator(context)
        self.secret_key = self.keygen.secret_key()
        self.public_key = self.keygen.create_public_key()
        self.galois_key = self.keygen.create_galois_keys()
        self.relin_keys = self.keygen.create_relin_keys()

        self.encryptor = seal.Encryptor(self.context, self.public_key)
        self.evaluator = seal.Evaluator(self.context)
        self.decryptor = seal.Decryptor(self.context, self.secret_key)

        self.encoder = seal.CKKSEncoder(self.context)
        print("all set")
        
    def encode(self, *args):
        return self.encoder.encode(*args)
    
    def decode(self, *args):
        return self.encoder.decode(*args)
    
    def encrypt(self, *args):
        return self.encryptor.encrypt(*args)

    def decrypt(self, *args):
        return self.decryptor.decrypt(*args)
    

In [27]:
poly_modulus_degrees = 8192

coeff_mod_bits = [40,40,40,40,40]

ck = scheme(poly_modulus_degrees, coeff_mod_bits, scheme="ckks")

all set


In [28]:
n_slot = ck.encoder.slot_count()
print("slot count", n_slot)

slot count 4096


In [29]:
vec = np.linspace(0, 1, n_slot)
print(vec)

[0.00000000e+00 2.44200244e-04 4.88400488e-04 ... 9.99511600e-01
 9.99755800e-01 1.00000000e+00]


In [30]:
scale = 2.**50

In [35]:
ptxt = ck.encode(vec, scale)
ctxt = ck.encrypt(ptxt)

In [36]:
rotated = ck.evaluator.rotate_vector(ctxt, 2, ck.galois_key)

In [37]:
rot_plain = ck.decrypt(rotated)

In [39]:
print(ck.decode(rot_plain))

[ 4.88400534e-04  7.32600647e-04  9.76800981e-04 ...  1.00000000e+00
 -1.78246307e-13  2.44200240e-04]


In [40]:
vec

array([0.00000000e+00, 2.44200244e-04, 4.88400488e-04, ...,
       9.99511600e-01, 9.99755800e-01, 1.00000000e+00])

Index가 두칸 옮겨짐. 

이걸 어디에 쓸까..? 
어째어째 잘 하면 slot-wise vector계산을 유지하면서 matrix 계산할 진행할 수 있음. 
좀 많이 복잡하겠지만?